In [67]:
import os
import subprocess
from pathlib import Path
import numpy as np
import operator
import json
import math
import resource

In [68]:
DIVISIONS = ['div1', 'div2', 'all']

DIV_THRESHOLD = 1650

In [69]:
division = 'div1'

In [70]:
time_limit = 10
memory_limit = 256 * 1024 * 1024

In [71]:
all_students = ['94101737', '94101975', '95105657', '95106494', '95108008', '95109242', '95109383', '96101262', '96101913', '96104881', '96105553', '96105575', '96105626', '96105637', '96105672', '96105683', '96105712', '96105807', '96105829', '96105853', '96105864', '96105875', '96105886', '96105948', '96105983', '96106033', '96106077', '96106117', '96109588', '96109599', '96109606', '96109663', '96109714', '96109736', '96109747', '96109758', '96109899', '96109906', '96110067', '96110129', '96110797', '97100581', '97102044', '97102441', '97105696', '97105771', '97105844', '97105855', '97106262', '97106295', '97106419', '97110155', '97110206', '97213093', '98209048']

In [72]:
import shutil

working_directory = os.path.abspath('')
os.chdir(working_directory)

submissions_path = Path(os.path.join(working_directory, 'submissions'))

tests_path = Path(os.path.join(working_directory, 'tests'))

executables_path = os.path.join(working_directory, f'executables-{division}')

rating_file = 'rating.json'

if os.path.exists(executables_path):
    shutil.rmtree(executables_path)

os.mkdir(executables_path)
    
outputs_path = Path(os.path.join(working_directory, f'outputs-{division}'))
    
if os.path.exists(outputs_path):
    shutil.rmtree(outputs_path)    
    
os.mkdir(outputs_path)

In [74]:
students = list(os.walk(submissions_path))[0][1]

if division != 'div1':
    students += ['dummy']

try:
    with open(rating_file, 'r', encoding='utf-8') as fp:
        rate = json.load(fp)

    for student in students:
        if student not in rate:
            print(f'FATAL ERROR: {student} has no rating!')

    for std in rate:
        if std not in students:
            print(f'{std} did not submit at this round.')
            students.append(std)
except FileNotFoundError:
    print('No rating file.')

if division == 'div1':
    students = list(filter(lambda x: rate[x] >= DIV_THRESHOLD, students))
elif division == 'div2':
    students = list(filter(lambda x: rate[x] < DIV_THRESHOLD, students))
            
final_results = {student: {'ranking': None, 'tests': {}, 'final_score': 0, 'prev_rating': 0, 'new_rating': 0} for student in students}

96106033 did not submit at this round.
96109736 did not submit at this round.
95109383 did not submit at this round.
97110155 did not submit at this round.
97213093 did not submit at this round.
dummy did not submit at this round.


In [75]:
print(division, 'contest')
print(f'{len(students)} students we have: {students}')

div1 contest
17 students we have: ['97102044', '97106419', '96109599', '94101737', '96105886', '96105626', '96105829', '97105844', '96105948', '96109906', '97105855', '97110206', '95108008', '96110067', '96109588', '96109747', '96109714']


In [76]:
print(submissions_path)
print(working_directory)
print(tests_path)
print(executables_path)

/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-3/judge/submissions
/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-3/judge
/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-3/judge/tests
/Users/kianooshabbasi/Dropbox/DA99/Covid Challenge/Round-3/judge/executables-div1


In [77]:
all_tests = list(map(lambda path: os.path.basename(path).split('in')[1].split('.txt')[0], tests_path.rglob('*.txt')))

In [78]:
print('Compiling codes...')

errors = 0

for student in students:
    cpp_files = list(Path(os.path.join(submissions_path, student)).rglob('*.c*'))
    ans = [] 
    for i in cpp_files:
        if i.name.endswith(".cxx") or i.name.endswith(".cpp"):
            ans.append(i)
    cpp_files = ans
    
    if len(cpp_files) != 1:
        print(f'Something wrong with {student}\'s files. Could not find the cpp file')
        final_results[student]['tests'] = {test: {'score': 0, 'cost': None, 'status': 'No Submission'} for test in all_tests}
        errors += 1
        continue

    cpp_file = cpp_files[0]
    output_file = os.path.join(submissions_path, student, 'output-runnable')
    
    process = subprocess.run(['g++', cpp_file, '-std=c++17', '-O2', '-o', output_file], capture_output=True)
    if process.returncode:
        print(f'Could not compile {student}\'s code.')
        print(f'*******************')
        print(process.stderr.decode()[:300])
        print(f'*******************')              
        final_results[student]['tests'] = {test: {'score': 0, 'cost': None, 'status': 'Compiler Error'} for test in all_tests}
        errors += 1
        continue
    
    
    shutil.move(output_file, os.path.join(executables_path, f'{student}.exec'))

print(f'Compilation finished for {len(students)} codes: {errors} error(s).')
print('\n=================================================\n')

Compiling codes...
Compilation finished for 17 codes: 0 error(s).




In [79]:
print('Executing codes...')

def limit_virtual_memory():
    resource.setrlimit(resource.RLIMIT_AS, (memory_limit, memory_limit))


for executable in list(Path(executables_path).rglob('*.exec')):    
    student_id = os.path.basename(executable).split('.exec')[0]
        
    print(f'Running {student_id}\'s codes')        
        
    for test_file in tests_path.rglob('*.txt'):
        test_name = os.path.basename(test_file).split('.txt')[0]
        
        test_num = test_name[2:]
        
        output_dir = os.path.join(outputs_path, test_name)
        os.makedirs(output_dir, exist_ok=True)
        
        output_path = os.path.join(output_dir, f'output-{student_id}.txt')
        
        input_file = open(test_file)
        output_file = open(output_path, 'w')
        
        try:
            subprocess.run([executable], stdin=input_file, stdout=output_file, timeout=time_limit, preexec_fn=limit_virtual_memory, check=True)
            output_file.close()
        except subprocess.TimeoutExpired:
            print(f'Time limit exceeded on test {test_name}')
            final_results[student_id]['tests'][test_num] = {'score': 0, 'cost': None, 'status': 'Time limit'}
            os.remove(output_path)
        except subprocess.CalledProcessError:
            print(f'Runtime error on test {test_name}')
            final_results[student_id]['tests'][test_num] = {'score': 0, 'cost': None, 'status': 'Runtime Error'}
            os.remove(output_path)
        
        input_file.close()
        
print('\n=================================================\n')

Executing codes...
Running 97110206's codes
Time limit exceeded on test input8
Time limit exceeded on test input13
Running 96109599's codes
Running 96105626's codes
Runtime error on test input1
Runtime error on test input25
Runtime error on test input19
Runtime error on test input18
Runtime error on test input24
Runtime error on test input2
Runtime error on test input3
Runtime error on test input23
Runtime error on test input22
Runtime error on test input4
Runtime error on test input20
Runtime error on test input21
Runtime error on test input5
Runtime error on test input10
Runtime error on test input11
Runtime error on test input9
Runtime error on test input13
Runtime error on test input12
Runtime error on test input16
Runtime error on test input17
Runtime error on test input15
Runtime error on test input14
Running 97105855's codes
Running 96109747's codes
Running 97106419's codes
Runtime error on test input1
Runtime error on test input2
Runtime error on test input3
Runtime error on te

In [80]:
checker_files = list(Path(working_directory).rglob('checker.cpp')) 
if len(checker_files) != 1:
    print("Please provide one checker.cpp file")

judge = os.path.join(working_directory, 'judge.exec')    

process = subprocess.run(['g++', checker_files[0], '-std=c++14', '-O2', '-o', judge], capture_output=True)
if process.returncode:
    print(f'Could not compile {student}\'s code:')
    print(process.stderr.decode()[:300])

In [81]:
cost = {}
for s in students:
    cost[s] = {}

for test in list(Path(outputs_path).rglob('in*')):
    this_test = os.path.basename(test) 
    test_num = this_test[2:]
    
    input_file = os.path.join(tests_path, this_test + '.txt')
    
    valid_submissions = list(test.rglob('output*.txt'))
    print("Seeing submissions output for test ", test_num, "\n")

    for student_output in valid_submissions:
        student_id = student_output.name.split('output-')[1].split('.txt')[0]
        
        try:
            out_path = os.path.join(test, f'checker-{student_id}.txt')
            log_path = os.path.join(test, f'log-{student_id}.txt')
            out_file = open(out_path, 'w')
            log_file = open(log_path, 'w')
            
            subprocess.run([judge, input_file, student_output], timeout=2, stdout=out_file, stderr=log_file, check=True)
            
            with open(out_path, "r") as f:
                checker_out = f.read()
            
            out_file.close()
            log_file.close()
            
            cost[student_id][test_num] = int(checker_out)
            print(f"{student_id}'s cost is {checker_out}")
        except subprocess.TimeoutExpired:
            print(f'Time limit exceeded on checker. its not normal')
            os.remove(log_file)
        except subprocess.CalledProcessError:
            print(f'Runtime error on checker. its not normal')
            os.remove(log_file)
        
        

Seeing submissions output for test  put16 

96110067's cost is 33916081905296073

97110206's cost is -1

96109906's cost is -1

95108008's cost is 122524070325429

96105829's cost is 33915685503560237

96109747's cost is 33880966093209197

96109588's cost is 33916606947221067

97106419's cost is 17266529038914293

97105844's cost is 181764708822965

96109599's cost is 33843877958274371

97105855's cost is 181764708822965

96105886's cost is 33915604282130678

96109714's cost is 33916081905296073

94101737's cost is 16973134115955863

Seeing submissions output for test  put11 

96110067's cost is 39546446476275276

97110206's cost is -1

96109906's cost is -1

95108008's cost is 254889427580668

96105829's cost is 39545857656425534

96109747's cost is 39544523698619475

96109588's cost is 39546446476275276

97105844's cost is 282522579247314

96109599's cost is 39505170329659602

97105855's cost is 282522579247314

96105886's cost is 39546111410353698

96109714's cost is 395464464762752

94101737's cost is 173237274228929

Seeing submissions output for test  put25 

96110067's cost is 11729045221907

97110206's cost is -1

96109906's cost is 5410779081857

95108008's cost is 485250348713

96105829's cost is 11678350740327

96109747's cost is 11625109908538

96109588's cost is 11731592391233

97106419's cost is 6045513677043

97105844's cost is 702408689125

96109599's cost is 11396183348750

97105855's cost is 735736000290

96105886's cost is 11721460135253

96109714's cost is 11729045221907

94101737's cost is 5863919967809

Seeing submissions output for test  put22 

96110067's cost is 6555994576459792

97110206's cost is -1

96109906's cost is -1

95108008's cost is 41197965338087

96105829's cost is 6555910579706482

96109747's cost is 6547101254826956

96109588's cost is 6560468103107512

97106419's cost is 3225996277223115

97105844's cost is 60959402213645

96109599's cost is 6532382955282228

97105855's cost is 61258996786741

96105886's cost is 655589719588952

In [84]:
def prob(ri, rj):
    delta = (rj - ri) / 400.
    return 1.0 / (1.0 + 10**(delta))

def return_student_ranking(all_scores, all_tests, test_weights, use_w = False):
    total_score = {student: 0 for student in students}
    for t in all_tests:
        results = []
        for s in students:
            if not t in all_scores[s]:
                continue
            res = all_scores[s][t]
            if res != -1:
                results.append((res, s))
        results.sort()
        first_three = results
        if len(results) > 3:
            first_three = first_three[:3]
        C = []
        for i in range(len(first_three)):
            C.append(first_three[i][0])
        Cm = np.mean(C)
        for s in students:
            if not t in all_scores[s]:
                continue
            res = all_scores[s][t]
            if res == -1:
                final_results[s]['tests'][t] = {'score': 0, 'cost': None, 'status': 'Invalid'}
                continue
            X = test_weights[t] * min(1, (Cm/res)**(1/3))
            final_results[s]['tests'][t] = {'score': round(X, 3), 'cost': res, 'status': 'Success'}
            total_score[s] += X
    sorted_d = sorted(total_score.items(), key=operator.itemgetter(1), reverse=True)
    rank = []
    eps = 1e-6
    for i, (student, score) in enumerate(sorted_d):
        cnt = 1
        for i2, (student2, score2) in enumerate(sorted_d):
            if score2 - eps > score:
                cnt += 1
        pair_rank = (student, cnt)
        rank.append(pair_rank)
        
        final_results[student]['final_score'] = round(score, 3)
        final_results[student]['ranking'] = cnt
    return rank

def get_new_ratings(rate, standings): # current rates, standing of student ids in latest round
    seed = {}
    for i in rate:
        seed[i] = 1
        for j in rate:
            if i == j:
                continue
            seed[i] += prob(rate[j], rate[i])
    new_rates = {}
    for idx, rank in enumerate(standings): # u: user
        lo, hi = 10, 4000
        u = rank[0]
        place = rank[1]
        m_i = math.sqrt((place) * seed[u])
        while(hi - lo > 1):
            mid = (hi + lo)//2
            new_seed = 1
            for j in rate:
                if j == u:
                    continue
                new_seed += prob(rate[j], mid)
            
            if new_seed < m_i: # over valued rating
                hi = mid
            else:
                lo = mid
        R = (hi + lo) * 0.5
        d_i = (R - rate[u]) * 0.5
        new_rates[u] = int(rate[u] + d_i)
    return new_rates


def update_ratings():
    rating_files = list(Path(working_directory).rglob(rating_file))
    rate = {}
    if len(rating_files) == 0:
        print("NO ratings will set every one to 1500")
        for s in students:
            rate[s] = 1500
    else:
        with open(rating_file, 'r', encoding='utf-8') as fp:
            rate = json.load(fp)
        rate = {key: value for key, value in rate.items() if key in final_results}
    print("previous rates -----")
    print(rate)
    for x, y in rate.items():
        final_results[x]['prev_rating'] = y
    standings = return_student_ranking(cost, all_tests, {t: 100 for t in all_tests}, False)
    print(standings)
    new_rates = get_new_ratings(rate, standings)
    destination = f"rating-{division}.json"
    print("new rates -----")
    print(new_rates)
    for x, y in new_rates.items():
        final_results[x]['new_rating'] = y
    with open(destination, 'w', encoding='utf-8') as fp:
        json.dump(new_rates, fp)

In [85]:
update_ratings()

previous rates -----
{'97105844': 1751, '97105855': 1724, '97110206': 1759, '95108008': 1744, '97102044': 1683, '97106419': 1677, '96109599': 1654, '94101737': 1654, '96105886': 1698, '96105626': 1654, '96105829': 1717, '96105948': 1659, '96109906': 1654, '96110067': 1680, '96109588': 1663, '96109747': 1691, '96109714': 1654}
[('95108008', 1), ('97105844', 2), ('97105855', 3), ('94101737', 4), ('96109599', 5), ('96109747', 6), ('96105829', 7), ('96105886', 8), ('96110067', 9), ('96109714', 9), ('96109588', 11), ('97106419', 12), ('96109906', 13), ('96105626', 14), ('97110206', 15), ('97102044', 16), ('96105948', 16)]
new rates -----
{'95108008': 1897, '97105844': 1850, '97105855': 1803, '94101737': 1734, '96109599': 1717, '96109747': 1727, '96105829': 1732, '96105886': 1706, '96110067': 1682, '96109714': 1663, '96109588': 1648, '97106419': 1648, '96109906': 1620, '96105626': 1610, '97110206': 1688, '97102044': 1614, '96105948': 1593}


In [86]:
data = sorted(final_results.items(), key= lambda item: item[1]['ranking'])

In [87]:
import xlsxwriter


def export_excel(data):

    workbook = xlsxwriter.Workbook(f'results-{division}.xlsx')
    worksheet = workbook.add_worksheet()
    cell_format1 = workbook.add_format({'bg_color': 'green'})
    cell_format2 = workbook.add_format({'bg_color': 'red'})

    col = 0
    worksheet.write(0, 0, "Ranking")
    worksheet.write(0, 1, "Student ID")
    worksheet.write(0, 2, "Final score")
    worksheet.write(0, 3, "Prev rating")
    worksheet.write(0, 4, "Rating change")
    worksheet.write(0, 5, "New rating")
    worksheet.write(0, 6, "")

    for i in range(1, len(all_tests) + 1):
        worksheet.write(0, i + 6, f"Test {i} score\n(cost)")
        
    for i in range(len(data)):
        col = 0
        worksheet.write(2*i+1, col, data[i][1]["ranking"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][0])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["final_score"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["prev_rating"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["new_rating"] - data[i][1]["prev_rating"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, data[i][1]["new_rating"])
        worksheet.write(2*i+2, col, "")
        col += 1
        worksheet.write(2*i+1, col, "")
        worksheet.write(2*i+2, col, "")
        col += 1
        for num in sorted(map(lambda x: x[3:], data[i][1]["tests"]), key=int):
            worksheet.write(2 * i + 1, col, data[i][1]["tests"][f"put{num}"]["score"])
            if data[i][1]["tests"][f"put{num}"]['status'] == "Success":
                worksheet.write(2 * i + 2, col, f'({data[i][1]["tests"][f"put{num}"]["cost"]})')
            else:
                worksheet.write(2 * i + 2, col, f'({data[i][1]["tests"][f"put{num}"]["status"]})')
        
            col += 1

    workbook.close()

In [88]:
export_excel(data)

In [89]:
os.remove(judge)

In [45]:
div2_standings = [
    (1,97100581),
    (2,96101262),
    (3,96109663),
    (4,97105771),
    (5,96110797),
    (6,96109606),
    (7,96105712),
    (7,96105983),
    (7,97105696),
    (10,96110129),
    (11,97102441),
    (12,96106077),
    (13,96105875),
    (14,98209048),
    (15,96105807),
    (16,96105683),
    (17,97106262),
    (18,96104881),
    (18,96105575),
    (18,96105553),
    (18,94170733),
    (18,97106295),
    (18,96106033),
    (18,96109736),
    (18,95109383),
    (18,97110155),
    (18,97213093),
    (18,'dummy'),
]
div2_standings = [(str(b), a) for a, b in div2_standings]
div2_students = [a for a, b in div2_standings]

In [58]:
def update_ratings_div2():
    with open('rating.json', 'r', encoding='utf-8') as fp:
        rate = json.load(fp)
    rate = {key: value for key, value in rate.items() if key in div2_students}
    print("previous rates -----")
    print(rate)
    print(div2_standings)
    new_rates = get_new_ratings(rate, div2_standings)
    destination = f"rating-div2.json"
    print("new rates -----")
    print(new_rates)
    with open(destination, 'w', encoding='utf-8') as fp:
        json.dump(new_rates, fp)

In [59]:
update_ratings_div2()

previous rates -----
{'97102441': 1589, '96109663': 1643, '97105696': 1562, '97100581': 1645, '96109606': 1564, '96105712': 1505, '96106077': 1550, '96104881': 1476, '96106033': 1470, '97105771': 1473, '96101262': 1478, '98209048': 1467, '96109736': 1488, '97106262': 1510, '97106295': 1440, '96105553': 1488, '96110129': 1427, '96105807': 1447, '96105575': 1417, '94170733': 1451, '95109383': 1407, '97110155': 1422, '96105683': 1463, '96110797': 1446, '96105875': 1457, '97213093': 1378, '96105983': 1388, 'dummy': 1373}
[('97100581', 1), ('96101262', 2), ('96109663', 3), ('97105771', 4), ('96110797', 5), ('96109606', 6), ('96105712', 7), ('96105983', 7), ('97105696', 7), ('96110129', 10), ('97102441', 11), ('96106077', 12), ('96105875', 13), ('98209048', 14), ('96105807', 15), ('96105683', 16), ('97106262', 17), ('96104881', 18), ('96105575', 18), ('96105553', 18), ('94170733', 18), ('97106295', 18), ('96106033', 18), ('96109736', 18), ('95109383', 18), ('97110155', 18), ('97213093', 18),

In [60]:
with open("rating-div2.json") as f:
    aaa = json.loads(f.read())

In [62]:
with open("rating-div21.json") as f:
    bbb = json.loads(f.read())

In [63]:
set(aaa.keys()) == set(bbb.keys())

True

In [64]:
with open('rating.json', 'r', encoding='utf-8') as fp:
    rate = json.load(fp)



In [66]:
for u in aaa:
    if aaa[u] != bbb[u]:
        print(u, aaa[u], bbb[u], rate[u])

96105983 1479 1473 1388
96105875 1473 1475 1457
97213093 1380 1383 1378
dummy 1376 1380 1373
